# Calculate median price

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sortedcontainers import SortedList
import numpy as np

In [2]:
nft_transactions_ = pd.read_csv('data/Data_API.csv',usecols=['Datetime_updated','Datetime_updated_seconds','Unique_id_collection','Collection_cleaned','Price_USD'])

In [3]:
nft_transactions = nft_transactions_.copy(deep=True)
nft_transactions.Datetime_updated_seconds = pd.to_datetime(nft_transactions.Datetime_updated_seconds)
nft_transactions.Datetime_updates = pd.to_datetime(nft_transactions.Datetime_updated)
nft_transactions.sort_values('Datetime_updated_seconds',inplace=True)

/var/folders/9t/frr8qwnd5h1_3mymt9gwrcm40000gn/T/ipykernel_49188/1025147062.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  nft_transactions.Datetime_updates = pd.to_datetime(nft_transactions.Datetime_updated)


Convert dataframe to dictionary for fast iteration

In [4]:
nfts = nft_transactions.to_dict('index')

Magic

In [5]:
def median(sorted_list):
    length = len(sorted_list)
    mid_index = length // 2
    if length % 2 == 0:
        mid_1 = sorted_list[mid_index][0]
        mid_2 = sorted_list[mid_index - 1][0]
        return (mid_1 + mid_2) / 2
    else:
        return sorted_list[mid_index][0]

collection_sales = {}
nft_values = {}
time_interval = pd.to_timedelta(7, unit='d')
for index, nft in tqdm(nfts.items()):
    price_usd = nft['Price_USD']
    collection_cleaned = nft['Collection_cleaned']
    datetime_updated_seconds = nft['Datetime_updated_seconds']
    if collection_cleaned not in collection_sales:
        collection_sales[collection_cleaned] = np.array([(0,0)]).reshape(1,2)
    if collection_sales[collection_cleaned].shape[0] > 1:
        median_price = np.median(collection_sales[collection_cleaned][:,0])
    else:
        median_price = 0
    
    nft_values[index] = {
       'median_price':median_price
    }
    collection_sales[collection_cleaned] = np.concatenate((collection_sales[collection_cleaned], [(price_usd, datetime_updated_seconds)]))
  
  #a = np.array(collection_sales[collection_cleaned])
  # index_list = np.where(a[:,1] < datetime_updated_seconds - time_interval)[0]
  # if index_list:
  #   index = index_list[-1]
  #   collection_sales[collection_cleaned] = a[index+1:]
  # for i in range(len(collection_sales[collection_cleaned]))[::-1]:
  #   pass
  # if collection_sales[collection_cleaned][i][1] < datetime_updated_seconds - time_interval:
  #   del collection_sales[collection_cleaned][i]
    
df_medianprice = pd.DataFrame.from_dict(nft_values, orient='index')
df_medianprice

  0%|          | 0/6071027 [00:00<?, ?it/s]

KeyboardInterrupt: 

Transfer the columns from the calculation to the original dataframe

In [ ]:
nft_transactions['median_price'] = df_medianprice['median_price']
nft_transactions

Check if the algorithm we used produces expected result on a collection

In [ ]:
nft_transactions[nft_transactions.Collection_cleaned == 'Cryptocelebritie'].loc[:,['Datetime_updated_seconds','median_price','Price_USD']]

These values are calculated on seconds precision. This leads to values being calculated with NFT transactions made seconds ago.  
We want to use daily resolution, which will lead to calculations being up to only the previous day of the NFT transaction.


In [ ]:
day_precision = nft_transactions.groupby(['Datetime_updated','Collection_cleaned']).first().reset_index()
day_precision

Merge the values with original dataframe on Datetime_updated and Collection_cleaned

In [ ]:
# remove old values
try:
    del nft_transactions['median_price']
except:
    pass
nft_transactions = nft_transactions.merge(day_precision.loc[:,['Datetime_updated','Collection_cleaned','median_price']], on=['Datetime_updated','Collection_cleaned'], how='left')
nft_transactions

Check results on the same collection

In [ ]:
nft_transactions[nft_transactions.Collection_cleaned == 'Cryptocelebritie'].loc[:,['Datetime_updated_seconds','median_price','Price_USD']]

# Visualize the values of top 10 collections

In [ ]:
top_collections = nft_transactions.groupby('Collection_cleaned').median_price.mean().sort_values()[-10:].index
nft_transactions[nft_transactions.Collection_cleaned.isin(top_collections)].groupby('Collection_cleaned').median_price.plot(legend=True, ylabel='n', figsize=(24,4))

In [ ]:
#nft_transactions.loc[:,['median_price']].to_csv('data/.csv')